In [1]:
from model import LyricsRNN
from data import LyricsDataset, padding_fn
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import re

device = torch.device('cpu')
checkpoint = torch.load('checkpoints/top5_embed/top5_embed-e00004.pt', map_location=device)
epoch = checkpoint['epoch']
all_losses = checkpoint['losses']
params = checkpoint['hyperparameters']

Data = LyricsDataset(params.input_file, vocab_file=params.vocab_file, vocab_size=params.vocab_size,
                     chunk_size=params.chunk_size, max_len=params.max_seq_len,
                     use_artist=params.use_artist)
print("%d batches per epoch" % (len(Data) / params.batch_size))
dataloader = DataLoader(Data, batch_size=params.batch_size, shuffle=True, num_workers=1, collate_fn=padding_fn,
                        drop_last=True)
ValData = LyricsDataset(re.sub('train', 'val', params.input_file), vocab_file=params.vocab_file,
                        chunk_size=params.chunk_size, use_artist=params.use_artist)
val_dataloader = DataLoader(ValData, batch_size=params.batch_size, num_workers=1, collate_fn=padding_fn, drop_last=True)

# Create model and optimizer
model = LyricsRNN(Data.vocab_len, Data.vocab_len, Data.PAD_ID, batch_size=params.batch_size, n_layers=params.n_layers,
                  hidden_size=params.hidden_size, word_embedding_size=params.word_embedding_size,
                  use_artist=params.use_artist, embed_artist=params.embed_artist, num_artists=Data.num_artists,
                  artist_embedding_size=params.artist_embedding_size
                  )
optimizer = torch.optim.Adam(model.parameters(), lr=params.learning_rate)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

Using vocab file: lyrics/top_5.vocab
chunking lyrics
6474 batches per epoch
Using vocab file: lyrics/top_5.vocab
chunking lyrics


LyricsRNN(
  (word_encoder): Embedding(7665, 128, padding_idx=0)
  (artist_encoder): Embedding(5, 32)
  (lstm): LSTM(160, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=7665, bias=True)
)

In [2]:
def generate(prime_str=[Data.START], artist=None, predict_len=100, temperature=0.8):
    inp = [Data.word2id(w) for w in prime_str]

    if type(artist) == str:
        artist = Data.artists.index(artist)

    predicted = model.evaluate(inp, artist, predict_len, temperature)

    predicted_words = [Data.id2word(w) for w in predicted]
    if Data.END in predicted_words:
        predicted_words = predicted_words[:predicted_words.index(Data.END) + 1]

    return ' '.join(predicted_words)
print(len(dataloader))

6474


In [3]:
val_loss = 0
loss_func = torch.nn.CrossEntropyLoss()
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        inp_seqs, inp_lens, out_seqs, out_lens, inp_artists, data = batch
        if params.use_artist:
            inp, target = [inp_seqs.to(device), inp_artists.to(device)], out_seqs.to(device)
        else:
            inp, target = inp_seqs.to(device), out_seqs.to(device)
        predictions = model(inp, inp_lens)
        loss = loss_func(predictions.view(-1, predictions.size(2)), target.view(-1).long())
        val_loss += loss.item()
        #loss = loss_func(predictions, target)
        #loss = model.loss(predictions, target)
        if i % 100 == 0:
            print({},i/len(dataloader))
    val_loss /= len(dataloader)
    print('ppl: {:5.2f},'.format(np.exp(val_loss)))

{} 0.0
{} 0.015446400988569663
{} 0.030892801977139325
{} 0.04633920296570899
{} 0.06178560395427865
{} 0.07723200494284832
{} 0.09267840593141798
{} 0.10812480691998765
{} 0.1235712079085573
{} 0.13901760889712697
{} 0.15446400988569664
{} 0.16991041087426628
{} 0.18535681186283595
{} 0.20080321285140562
{} 0.2162496138399753
{} 0.23169601482854496
{} 0.2471424158171146
{} 0.2625888168056843
{} 0.27803521779425394
{} 0.2934816187828236
{} 0.3089280197713933
{} 0.3243744207599629
{} 0.33982082174853256
{} 0.35526722273710226
{} 0.3707136237256719
{} 0.3861600247142416
{} 0.40160642570281124
{} 0.4170528266913809
{} 0.4324992276799506
{} 0.4479456286685202
{} 0.4633920296570899
{} 0.47883843064565956
{} 0.4942848316342292
{} 0.5097312326227988
{} 0.5251776336113686
{} 0.5406240345999382
{} 0.5560704355885079
{} 0.5715168365770775
{} 0.5869632375656472
{} 0.6024096385542169
{} 0.6178560395427866
{} 0.6333024405313562
{} 0.6487488415199258
{} 0.6641952425084955
{} 0.6796416434970651
{} 0.

In [4]:
from model import LyricsRNN
from data import LyricsDataset, padding_fn
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import re

device = torch.device('cpu')
checkpoint = torch.load('checkpoints/top5_onehot/top5_onehot-e00010.pt', map_location=device)
epoch = checkpoint['epoch']
all_losses = checkpoint['losses']
params = checkpoint['hyperparameters']

Data = LyricsDataset(params.input_file, vocab_file=params.vocab_file, vocab_size=params.vocab_size,
                     chunk_size=params.chunk_size, max_len=params.max_seq_len,
                     use_artist=params.use_artist)
print("%d batches per epoch" % (len(Data) / params.batch_size))
dataloader = DataLoader(Data, batch_size=params.batch_size, shuffle=True, num_workers=1, collate_fn=padding_fn,
                        drop_last=True)
ValData = LyricsDataset(re.sub('train', 'val', params.input_file), vocab_file=params.vocab_file,
                        chunk_size=params.chunk_size, use_artist=params.use_artist)
val_dataloader = DataLoader(ValData, batch_size=params.batch_size, num_workers=1, collate_fn=padding_fn, drop_last=True)

# Create model and optimizer
model = LyricsRNN(Data.vocab_len, Data.vocab_len, Data.PAD_ID, batch_size=params.batch_size, n_layers=params.n_layers,
                  hidden_size=params.hidden_size, word_embedding_size=params.word_embedding_size,
                  use_artist=params.use_artist, embed_artist=params.embed_artist, num_artists=Data.num_artists,
                  artist_embedding_size=params.artist_embedding_size
                  )
optimizer = torch.optim.Adam(model.parameters(), lr=params.learning_rate)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()
def generate(prime_str=[Data.START], artist=None, predict_len=100, temperature=0.8):
    inp = [Data.word2id(w) for w in prime_str]

    if type(artist) == str:
        artist = Data.artists.index(artist)

    predicted = model.evaluate(inp, artist, predict_len, temperature)

    predicted_words = [Data.id2word(w) for w in predicted]
    if Data.END in predicted_words:
        predicted_words = predicted_words[:predicted_words.index(Data.END) + 1]

    return ' '.join(predicted_words)
print(len(dataloader))
val_loss = 0
loss_func = torch.nn.CrossEntropyLoss()
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        inp_seqs, inp_lens, out_seqs, out_lens, inp_artists, data = batch
        if params.use_artist:
            inp, target = [inp_seqs.to(device), inp_artists.to(device)], out_seqs.to(device)
        else:
            inp, target = inp_seqs.to(device), out_seqs.to(device)
        predictions = model(inp, inp_lens)
        loss = loss_func(predictions.view(-1, predictions.size(2)), target.view(-1).long())
        val_loss += loss.item()
        #loss = loss_func(predictions, target)
        #loss = model.loss(predictions, target)
        if i % 100 == 0:
            print({},i/len(dataloader))
    val_loss /= len(dataloader)
    print('ppl: {:5.2f},'.format(np.exp(val_loss)))

Using vocab file: lyrics/top_5.vocab
chunking lyrics
6474 batches per epoch
Using vocab file: lyrics/top_5.vocab
chunking lyrics
6474
{} 0.0
{} 0.015446400988569663
{} 0.030892801977139325
{} 0.04633920296570899
{} 0.06178560395427865
{} 0.07723200494284832
{} 0.09267840593141798
{} 0.10812480691998765
{} 0.1235712079085573
{} 0.13901760889712697
{} 0.15446400988569664
{} 0.16991041087426628
{} 0.18535681186283595
{} 0.20080321285140562
{} 0.2162496138399753
{} 0.23169601482854496
{} 0.2471424158171146
{} 0.2625888168056843
{} 0.27803521779425394
{} 0.2934816187828236
{} 0.3089280197713933
{} 0.3243744207599629
{} 0.33982082174853256
{} 0.35526722273710226
{} 0.3707136237256719
{} 0.3861600247142416
{} 0.40160642570281124
{} 0.4170528266913809
{} 0.4324992276799506
{} 0.4479456286685202
{} 0.4633920296570899
{} 0.47883843064565956
{} 0.4942848316342292
{} 0.5097312326227988
{} 0.5251776336113686
{} 0.5406240345999382
{} 0.5560704355885079
{} 0.5715168365770775
{} 0.5869632375656472
{} 

In [5]:
from model import LyricsRNN
from data import LyricsDataset, padding_fn
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import re

device = torch.device('cpu')
checkpoint = torch.load('checkpoints/dolly/dolly-e00010.pt', map_location=device)
epoch = checkpoint['epoch']
all_losses = checkpoint['losses']
params = checkpoint['hyperparameters']

Data = LyricsDataset(params.input_file, vocab_file=params.vocab_file, vocab_size=params.vocab_size,
                     chunk_size=params.chunk_size, max_len=params.max_seq_len,
                     use_artist=params.use_artist)
print("%d batches per epoch" % (len(Data) / params.batch_size))
dataloader = DataLoader(Data, batch_size=params.batch_size, shuffle=True, num_workers=1, collate_fn=padding_fn,
                        drop_last=True)
ValData = LyricsDataset(re.sub('train', 'val', params.input_file), vocab_file=params.vocab_file,
                        chunk_size=params.chunk_size, use_artist=params.use_artist)
val_dataloader = DataLoader(ValData, batch_size=params.batch_size, num_workers=1, collate_fn=padding_fn, drop_last=True)

# Create model and optimizer
model = LyricsRNN(Data.vocab_len, Data.vocab_len, Data.PAD_ID, batch_size=params.batch_size, n_layers=params.n_layers,
                  hidden_size=params.hidden_size, word_embedding_size=params.word_embedding_size,
                  use_artist=params.use_artist, embed_artist=params.embed_artist, num_artists=Data.num_artists,
                  artist_embedding_size=params.artist_embedding_size
                  )
optimizer = torch.optim.Adam(model.parameters(), lr=params.learning_rate)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()
def generate(prime_str=[Data.START], artist=None, predict_len=100, temperature=0.8):
    inp = [Data.word2id(w) for w in prime_str]

    if type(artist) == str:
        artist = Data.artists.index(artist)

    predicted = model.evaluate(inp, artist, predict_len, temperature)

    predicted_words = [Data.id2word(w) for w in predicted]
    if Data.END in predicted_words:
        predicted_words = predicted_words[:predicted_words.index(Data.END) + 1]

    return ' '.join(predicted_words)
print(len(dataloader))
val_loss = 0
loss_func = torch.nn.CrossEntropyLoss()
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        inp_seqs, inp_lens, out_seqs, out_lens, inp_artists, data = batch
        if params.use_artist:
            inp, target = [inp_seqs.to(device), inp_artists.to(device)], out_seqs.to(device)
        else:
            inp, target = inp_seqs.to(device), out_seqs.to(device)
        predictions = model(inp, inp_lens)
        loss = loss_func(predictions.view(-1, predictions.size(2)), target.view(-1).long())
        val_loss += loss.item()
        #loss = loss_func(predictions, target)
        #loss = model.loss(predictions, target)
        if i % 100 == 0:
            print({},i/len(dataloader))
    val_loss /= len(dataloader)
    print('ppl: {:5.2f},'.format(np.exp(val_loss)))

Using vocab file: lyrics/dolly-parton.vocab
chunking lyrics
798 batches per epoch
Using vocab file: lyrics/dolly-parton.vocab
chunking lyrics
798
{} 0.0
{} 0.12531328320802004
{} 0.2506265664160401
{} 0.37593984962406013
{} 0.5012531328320802
{} 0.6265664160401002
{} 0.7518796992481203
{} 0.8771929824561403
ppl: 24.37,
